# Workspaces - Export Snapshot

**Author**: Melissa Coates  

**Last updated and tested**: August 22, 2022 

**Purpose:** This notebook contains two examples to export workspace artifact metadata:

- Example 1 - using the PowerShell cmdlet
- Example 2 - using the API

**Type of authentication**: You can log in with either:

- A domain user account (requires Power BI administrator permissions), or
- A service principal (with permission to run admin APIs)

**How data is being accessed:** The notebook uses:

- Both examples - Power BI Management Module for authentication: Connect-PowerBIServiceAccount
- Example 1 - Power BI Management Module for extracting the workspace information: Get-PowerBIWorkspace
- Example 2 - Power BI Management Module for calling the API: Invoke-PowerBIRestMethod --plus-- the Power BI Admin API: GetGroupsAsAdmin

**Type of script**: Interactive, ad hoc (non-scheduled). This notebook is a guided learning experience to get familiar with how to query the workspace metadata. It's not production-ready. 

More information, including prerequisites, is included at the bottom of this notebook.

## **Log in to the Power BI Service**
---
**Update before executing:**

- **Type of authentication:** 
    - **Line 2 - Choice for type of auth:** Input either '**User**' or '**SP**' without the quotes. This tells the script whether or not to log in as a user or a service principal.
- **If using user authentication:**
    - **Line 3 - Email address:** Input the email address for the user. This user must be a Power BI administrator. MFA can't be enabled for this account.
- **If using service principal authentication:**
    - **Line 4 - Azure AD app ID**: Input the app ID (aka client ID). 
    - **Line 5 - Azure tenant ID**: Input the tenant ID (aka directory ID).

**Interactive prompts:**

The script will interactively prompt you for the sensitive information (because hard-coding of passwords and secrets isn't secure):

- **If using user authentication:**
    - It will ask for the user's password.
- **If using service principal authentication:**
    - It will ask for the app secret.

**Comments:**

- This script uses the PowerShell cmdlet: [<u>Connect-PowerBIServiceAccount</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.profile/connect-powerbiserviceaccount).
- After signing in, the access token is active for one hour.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$TypeOfAuth = 'User'  #Options: User or SP
[string]$DomainUserEmailAddr = 'user@domain.com'
[string]$AzureADAppID = '1234-1234'
[string]$AzureTenantID = '1234-1234'
#--------------------------------------------

if ($TypeOfAuth -eq 'User')
{
    Write-Verbose "Domain user authentication will be used." -Verbose
    [securestring]$DomainUserPW = Read-Host -Prompt "Input password for $DomainUserEmailAddr" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($DomainUserEmailAddr, $DomainUserPW)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj
}
elseif ($TypeOfAuth -eq 'SP')
{
    Write-Verbose "Service principal authentication will be used." -Verbose
    [securestring]$AzureADAppSecret = Read-Host -Prompt "Input secret for $AzureADAppID" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($AzureADAppID, $AzureADAppSecret)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj -ServicePrincipal -Tenant $AzureTenantID
}
else 
{
    Write-Verbose "There's a problem. The Type of Auth variable input was $TypeOfAuth. Expected values are User or SP." -Verbose
}

## **Example 1 - Export Workspace Info with Artifacts: Using Cmdlet**
---
The next script will export the workspace metadata for the entire tenant, including the artifacts contained within the workspace. It's a snapshot as of a point in time.

**Comments:**

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIWorkspace</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.workspaces/get-powerbiworkspace)
- **Permissions required**: Power BI administrator permissions, or a service principal with permission to use read-only admin APIs, are required for the organization scope.

**Update before executing:**

- **Line 2 - Path for export file.** This folder needs to already exist.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$ExportFileLocation = 'C:\Power-BI-Raw-Data\Workspace-Metadata'
#--------------------------------------------

[string]$DateTimeFileWrittenUTCLabel = ([datetime]::Now.ToUniversalTime()).ToString("yyyyMMddHHmm")  #Snapshot date when data was extracted
[string]$ExportFileName = 'AllWorkspaceMetadata-FromCmdlet-' + $DateTimeFileWrittenUTCLabel + '.json' #FromCmdlet in the name since the another example does the same thing using the API directly

Get-PowerBIWorkspace -Scope Organization -Include All | ConvertTo-JSON -Depth 10 | Out-File "$ExportFileLocation\$ExportFileName"

Write-Verbose "File written: $ExportFileName" -Verbose

## **Example 2 - Export Workspace Info with Artifacts: Using REST API Directly**
---
The next script is the same as the above - it exports the workspace metadata for the entire tenant, including the artifacts contained within the workspace. It's a snapshot as of a point in time.

However, this script calls the admin 'Groups' REST API to get the data, instead of using the PowerShell cmdlet directly. 

**Comments:**

- **API used:** This script uses the admin API: [<u>GetGroupsAsAdmin</u>](https://docs.microsoft.com/en-us/rest/api/power-bi/admin/groups-get-groups-as-admin).
- **How API is called:** The API is called via the [<u>Invoke-PowerBIRestMethod</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.profile/invoke-powerbirestmethod) cmdlet from the Power BI Management Module. 
- **How the Power BI Management Module is being used:**
    - Authentication (simpler than OAuth2 authentication).
    - Calling the API (the Invoke-PowerBIRestMethod cmdlet is like a 'wrapper' around the API call).
- **Permissions required**: Power BI administrator permissions, or a service principal with permission to use read-only admin APIs, are required for the organization scope.
- **Types of artifacts included:** The $expand parameter specifies which objects are included.
- **Encoding of the URL:** The %24 shown in the URL is because the $ sign is reserved by PowerShell for variables & parameters. For this reason, the $ sign needs to be replaced in PowerShell scripts with a %24. The URL knows that it represents the $ character. Note that HTPP examples shown online will show the $.

**Update before executing:**

- **Line 2 - Path for export file.** This folder needs to already exist.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$ExportFileLocation = 'C:\Power-BI-Raw-Data\Workspace-Metadata'
#--------------------------------------------

[int32]$NbrOfRecordsToFetchPerBatch = 1000 
[int32]$NbrOfRecordsToSkip = 0 

[string]$DateTimeFileWrittenUTCLabel = ([datetime]::Now.ToUniversalTime()).ToString("yyyyMMddHHmm")  #Snapshot date when data was extracted
[string]$ExportFileName = 'AllWorkspaceMetadata-FromAPI-' + $DateTimeFileWrittenUTCLabel + '.json' #FromAPI in the name since the previous example did the same thing using the cmdlet

[string]$URL = "https://api.powerbi.com/v1.0/myorg/admin/Groups?%24top=$NbrOfRecordsToFetchPerBatch&%24skip=$NbrOfRecordsToSkip&%24expand=datasets,dataflows,reports,dashboards,workbooks,users"

Invoke-PowerBIRestMethod -Url $URL -Method GET | Out-File "$ExportFileLocation\$ExportFileName"

Write-Verbose "File written: $ExportFileName" -Verbose

## **Observations**

* * *

When you compare the two files that were exported:

- **Readability**: The data exported from the cmdlet is formatted with line breaks, so it's more human readable than the data exported from the REST API.
- **Data**: If new data is made available, it'll be available in the API before it's available to the cmdlet. 
- **Use of PowerShell Modules:** In the 2nd example above, the PowerShell Management Module is only used for authentication and to call the APIs. By using the Invoke-PowerBIRestMethod cmdlet, you don't have to build the body and the headers. Authentication is easier, and calling the API is easier. 
- **Tool/Language Flexibility**: Many organizations choose to call the APIs directly even if a cmdlet is available. This gives them flexibility to change tools or languages over time, and reduces their dependency on the PowerShell Management Module.

Large tenants:

- The GetGroupsAsAdmin API has a [<u>maximum # of requests per hour</u>](https://docs.microsoft.com/en-us/rest/api/power-bi/admin/groups-get-groups-as-admin#limitations). 
- In a large tenant with a lot of data, it's much more efficient to use the [<u>Scanner APIs</u>](https://docs.microsoft.com/en-us/power-bi/enterprise/service-admin-metadata-scanning) instead.
- Consider the above to be a learning exercise. If your tenant is a large one, convert to using the Scanner APIs before you operationalize and schedule your scripts.

## **More Information About This File**  
---
## <span style="font-size: 14px;">Prerequisites for running this script:</span>  

- The [Power BI Management module](https://www.powershellgallery.com/packages/MicrosoftPowerBIMgmt) for PowerShell needs to be installed.
    
- The [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension needs to be installed and enabled in VS Code and/or Azure Data Studio.
    
- Either a user account with Power BI administrator permissions, or a service principal.
    

This script **interactively prompts** for the sensitive information to be passed into a credential object. Therefore, it's only useful for ad hoc purposes. A different technique is used for securely storing the credentials when a script is to be operationalized and scheduled.

The original / most recent version of this notebook can be found on Melissa Coates' [Power-BI-Admin repository in GitHub](https://github.com/sqlchick/Power-BI-Admin).